In [1]:
# ----------------------------------------------------------------------
# 라이브러리 및 데이터 준비
# ----------------------------------------------------------------------
import numpy as np
import pandas as pd
from sklearn.svm import OneClassSVM # One-Class SVM 함수
from sklearn.preprocessing import StandardScaler

# 1. 정상 데이터 (Normal Data): 100개 포인트가 2 주변에 집중
rng = np.random.RandomState(42)
X_normal = 0.3 * rng.randn(100, 2)
X_train = X_normal + 2 

# 2. 테스트 데이터 (정상 데이터 + 이상치)
# 정상 데이터 100개와 확실한 이상치 20개
X_outliers = rng.uniform(low=-4, high=4, size=(20, 2))
X_test = np.r_[X_train, X_outliers] 

df = pd.DataFrame(X_test, columns=['Feature1', 'Feature2'])
print("--- 테스트 데이터셋 (정상 + 이상치) ---")
print(df.head())

--- 테스트 데이터셋 (정상 + 이상치) ---
   Feature1  Feature2
0  2.149014  1.958521
1  2.194307  2.456909
2  1.929754  1.929759
3  2.473764  2.230230
4  1.859158  2.162768


In [2]:
# ----------------------------------------------------------------------
#  2단계: 데이터 스케일링 및 모델 학습
# ----------------------------------------------------------------------
# 데이터 스케일링 (PCA와 유사하게 성능 향상에 필수적)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # 학습 데이터(정상)에 fit 및 transform
X_test_scaled = scaler.transform(X_test)       # 테스트 데이터에 transform만 적용

# One-Class SVM 모델 생성 및 학습
# nu: 이상치 비율의 상한선 (전체 학습 데이터 중 허용할 이상치의 비율)
# gamma: 커널 함수의 계수 (경계의 복잡성 조절)
model = OneClassSVM(kernel="rbf", nu=0.1, gamma=0.1) 

# 모델 학습: 오직 '정상 패턴'을 정의하기 위해 X_train_scaled만을 사용
model.fit(X_train_scaled)

print("\n--- 2단계: 정상 데이터 패턴 학습 완료 ---")


--- 2단계: 정상 데이터 패턴 학습 완료 ---


In [3]:
# ----------------------------------------------------------------------
# 3단계: 이상치 예측 및 점수 확인
# ----------------------------------------------------------------------

# 1) 예측: 각 데이터 포인트가 정상 경계 내에 있는지 예측
#    결과: 1 (정상, Inlier), -1 (이상치, Outlier)
predictions = model.predict(X_test_scaled)

# 2) 이상치 점수 (Anomaly Score) 계산
#    점수가 0보다 작을수록 경계(Decision Boundary)에서 멀리 떨어져 이상치일 확률이 높음.
scores = model.decision_function(X_test_scaled)

# 결과를 DataFrame에 추가
df['anomaly_prediction'] = predictions
df['anomaly_score'] = scores

print("\n--- 3단계: 예측 및 점수 결과 (상위 5개) ---")
print(df.head())

# ----------------------------------------------------------------------
#  4단계: 탐지된 이상치 확인
# ----------------------------------------------------------------------

# 예측값이 -1인 데이터만 필터링하여 이상치 확인
outliers = df[df['anomaly_prediction'] == -1].sort_values(by='anomaly_score')

print("\n--- 4단계: 탐지된 이상치 확인 (점수가 낮은 순 = 가장 이상함) ---")
# 점수가 가장 낮은(음수 폭이 큰) 데이터가 모델 경계에서 가장 멀리 떨어진 이상치임.
print(outliers.head())


--- 3단계: 예측 및 점수 결과 (상위 5개) ---
   Feature1  Feature2  anomaly_prediction  anomaly_score
0  2.149014  1.958521                   1       1.111368
1  2.194307  2.456909                   1       0.532634
2  1.929754  1.929759                   1       1.114341
3  2.473764  2.230230                   1       0.223374
4  1.859158  2.162768                   1       1.036433

--- 4단계: 탐지된 이상치 확인 (점수가 낮은 순 = 가장 이상함) ---
     Feature1  Feature2  anomaly_prediction  anomaly_score
113 -3.755998 -3.701214                  -1      -4.599079
118 -3.586546  0.250837                  -1      -4.599079
109 -0.995336 -3.248144                  -1      -4.599079
110  0.626241 -3.712462                  -1      -4.599079
115 -2.983516  0.177946                  -1      -4.599079


In [4]:
#
# #  One-Class SVM 이상치 탐지 결과 해석
#
# # One-Class SVM (OCSVM) 원리:
# # OCSVM은 정상 데이터(X_train)만 사용하여 데이터가 존재하는 영역 주위에 **단일 경계(Boundary)**를 설정합니다.
# # 테스트 데이터가 이 경계 안에 있으면 정상(1), 밖에 있으면 이상치(-1)로 분류합니다.
#
# # --- 3단계: 예측 및 점수 결과 (상위 5개) ---
# # (이 데이터는 X_train에서 온, 경계 내부에 있는 정상 데이터입니다.)
#
# # Column 해석:
# # anomaly_prediction: 모델 경계 내부에 있는지 예측한 결과입니다. (1: 정상, -1: 이상치)
# # anomaly_score (Decision Function): 경계(0)로부터의 거리입니다.
# #     - **양수 (0 초과):** 경계 내부에 있어 정상으로 분류됨.
# #     - **음수 (0 미만):** 경계 외부에 있어 이상치로 분류됨.
# #     - **값이 클수록 (더 양수일수록):** 경계 중앙에 가까워 확실한 정상.
# #     - **값이 작을수록 (더 음수일수록):** 경계에서 멀리 떨어져 확실한 이상치.
#
#    Feature1  Feature2  anomaly_prediction  anomaly_score
# 0  2.149014  1.958521                   1       1.111368 # 점수가 양수(1.11)이고 큼. 경계 내부에 깊숙이 위치한 확실한 정상 데이터(1)입니다.
# 1  2.194307  2.456909                   1       0.532634 # 점수가 양수(0.53). 경계 안쪽에 있지만, 0번 샘플보다는 경계에 더 가까움.
# 2  1.929754  1.929759                   1       1.114341 # 점수가 가장 높음(1.114). 경계 중앙에 가장 가까운 확실한 정상 데이터(1)입니다.
# 3  2.473764  2.230230                   1       0.223374 # 점수가 0.22로 양수이지만 0에 가장 가까움. 경계선 바로 안쪽에 위치한 데이터(1)일 가능성이 높습니다.
# 4  1.859158  2.162768                   1       1.036433 # 점수가 양수(1.03). 확실한 정상 데이터(1)입니다.
#
# # ----------------------------------------------------------------------
#
# # --- 4단계: 탐지된 이상치 확인 (점수가 낮은 순 = 가장 이상함) ---
# # (예측값(-1)을 기준으로 필터링되었으며, 점수(음수)가 가장 작은 순서로 정렬됨.)
#
# # Column 해석:
# # anomaly_score: 모두 음수이며, 이 음수 값의 **절댓값이 클수록** (즉, 값이 더 낮을수록) **경계에서 가장 멀리** 떨어진 강력한 이상치입니다.
#
#      Feature1  Feature2  anomaly_prediction  anomaly_score
# 118 -3.586546  0.250837                  -1      -4.599079 # 이상치(-1)로 분류됨. 점수가 -4.59로 매우 낮아, 모델이 정의한 정상 경계에서 가장 멀리 떨어진 데이터입니다.
# 113 -3.755998 -3.701214                  -1      -4.599079 # 다른 이상치와 동일한 점수를 가짐. 이들은 경계 밖의 동일한 거리에 위치한다고 해석할 수 있습니다.
# 109 -0.995336 -3.248144                  -1      -4.599079 # 동일한 점수를 가짐.
# 110  0.626241 -3.712462                  -1      -4.599079 # 동일한 점수를 가짐.
# 115 -2.983516  0.177946                  -1      -4.599079 # 동일한 점수를 가짐.
#
# # 결론: OCSVM은 훈련된 정상 데이터 군집에서 멀리 떨어진 모든 테스트 포인트를 이상치(-1)로 잘 감지했으며, 이들은 모두 경계에서 유사하게 멀리 떨어져 있는 것으로 나타났습니다.
#